## Data fetching notebook

### Imports

In [1]:
import os
import re 
import wget # for downloading files in python
import tweepy 
from textblob import TextBlob 

In [2]:
os.chdir('..')
from data_fetching.keys import API_KEY, API_SECRET_KEY, ACCESS_TOKEN, ACCESS_TOKEN_SECRET

### Parameters

In [25]:
TEST_SCREEN_NAME = 'NatGeo'

DATA_SAVE_LOC = os.getcwd()+"\\data\\images-test\\"

In [26]:
# Create and authenticate an API object
auth = tweepy.OAuthHandler(API_KEY, API_SECRET_KEY)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
api = tweepy.API(auth)

In [27]:
user = api.me()
user.screen_name

'OllieSmith4'

In [28]:
example = tweepy.Cursor(api.search, q='@'+TEST_SCREEN_NAME, show_user=False).items(15)

for tweet in example:

    print(tweet.text, tweet.id_str, tweet.created_at)

RT @NatGeo: A new image, which shows a stream of plasma between two galaxy clusters, is the first known sighting of its kind https://t.co/4… 1137123044890165248 2019-06-07 22:23:55
RT @NatGeo: Yellowstone National Park's geysers and geothermal features are fascinating to watch—and listen to https://t.co/mG3pCdlL0K 1137122927118299136 2019-06-07 22:23:27
RT @NatGeo: What is the moon made of, and how did it form? Learn all about the moon's origins and how humans first explored Earth's only na… 1137122908831191040 2019-06-07 22:23:22
RT @elOrdenMundial: La expansión de Shangai en los últimos 200 años es abismal. Relativamente tranquilo hasta los años 80, cuando el gobier… 1137122816837529601 2019-06-07 22:23:00
@CamiloPrietoVal @UN @NatGeo ya actualizó la cifra, al día de hoy ya son 9 millones 😕 1137122734897532931 2019-06-07 22:22:41
@NatGeo Can you help us figure out which part of Mars is the Moon?

Asking for a friend. 1137122603947253760 2019-06-07 22:22:10
RT @NatGeo: An ancient lake

In [29]:
def get_url_from_tweet_id(tweet_id):
    return 'https://twitter.com/statuses/'+str(tweet_id)

In [30]:
tweets = api.user_timeline(screen_name=TEST_SCREEN_NAME,
                           count=200, include_rts=False,
                           exclude_replies=True)

In [31]:
media_files = set()
for tweet in tweets:
    media = tweet.entities.get('media', [])
    if(len(media) > 0):
        media_files.add(media[0]['media_url'])

In [32]:
media_files

{'http://pbs.twimg.com/amplify_video_thumb/1136644676503707649/img/7ogYoBTvuHOdy3FM.jpg',
 'http://pbs.twimg.com/amplify_video_thumb/956302544837468161/img/WNdgcEDLH3rRIh93.jpg',
 'http://pbs.twimg.com/amplify_video_thumb/957002280754130946/img/WbuIb-lgK_5rCJsP.jpg',
 'http://pbs.twimg.com/amplify_video_thumb/958831625399390208/img/jm6EuhLPJrs2KChc.jpg',
 'http://pbs.twimg.com/amplify_video_thumb/962001378947354624/img/gDkOz9On2cLZyImv.jpg',
 'http://pbs.twimg.com/amplify_video_thumb/966435600860508160/img/YW03SSYzks6PS-Zr.jpg',
 'http://pbs.twimg.com/amplify_video_thumb/966805094581891073/img/sJr3Ujo5MmEMExk6.jpg',
 'http://pbs.twimg.com/amplify_video_thumb/969649205777256448/img/95SHjeclK1OgHO6c.jpg',
 'http://pbs.twimg.com/amplify_video_thumb/971423524542808065/img/VqAnIqwPA65OxkZ0.jpg',
 'http://pbs.twimg.com/amplify_video_thumb/987448056554643456/img/fGN1rYISZ7PfOyt_.jpg',
 'http://pbs.twimg.com/media/D7MsTlXXkAAnPao.jpg',
 'http://pbs.twimg.com/media/D7NAi7TW0AAqp-0.jpg',
 'http:

In [33]:
current_downloaded_files = os.listdir(DATA_SAVE_LOC)

In [34]:
current_downloaded_files

['D6orhLEXsAAcdWj.jpg']

In [35]:
for media_file in media_files:
    if media_file.split('/')[-1] in current_downloaded_files:
        print('File already downloaded: {}'.format(media_file.split('/')[-1]))

In [36]:
for media_file in media_files:
    if media_file.split('/')[-1] in current_downloaded_files:
        print('File already downloaded: {}'.format(media_file.split('/')[-1]))
    else:
        wget.download(media_file, DATA_SAVE_LOC)

100% [............................................................................] 164098 / 164098

In [37]:
os.listdir(DATA_SAVE_LOC)

['7ogYoBTvuHOdy3FM.jpg',
 '95SHjeclK1OgHO6c.jpg',
 'D6orhLEXsAAcdWj.jpg',
 'D7jl4jKW0AYo-gk.jpg',
 'D7MsTlXXkAAnPao.jpg',
 'D7NAi7TW0AAqp-0.jpg',
 'D7sA50UXYAAOj_j.jpg',
 'D7wo2qEW4AAGwwK.jpg',
 'D8e-7rOXkAA7iL0.jpg',
 'D8HpAw4XkAAFeLV.jpg',
 'fGN1rYISZ7PfOyt_.jpg',
 'gDkOz9On2cLZyImv.jpg',
 'jm6EuhLPJrs2KChc.jpg',
 'sJr3Ujo5MmEMExk6.jpg',
 'VqAnIqwPA65OxkZ0.jpg',
 'WbuIb-lgK_5rCJsP.jpg',
 'WNdgcEDLH3rRIh93.jpg',
 'YW03SSYzks6PS-Zr.jpg']

## Let's try neural style transfer. 

Pretrain a model which has a certain style attached to it, and try putting a bunch of images into that style